In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import Select

import pandas as pd
from bs4 import BeautifulSoup
import requests

class Navegador:
    def __init__(self):
        # Configurar opções do Chrome
        options = Options()
        options.add_argument("--enable-automation")
        options.add_argument("--start-maximized")
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-popup-blocking")
        options.add_argument("--kiosk-printing")


        # Inicializar o WebDriver do Chrome com as opções configuradas
        #self.driver = webdriver.Remote(command_executor="http://localhost:4444/wd/hub", options=options)
        self.driver = webdriver.Chrome(options=options)
        self.wait = WebDriverWait(self.driver, 10)
        self.by = By
        self.locator = {
            "XPATH": By.XPATH,
            "ID": By.ID,
            "CLASS_NAME": By.CLASS_NAME,
            "LINK_TEXT": By.LINK_TEXT,
            "NAME": By.NAME,
            "PARTIAL_LINK_TEXT": By.PARTIAL_LINK_TEXT,
            "TAG_NAME": By.TAG_NAME
        }

    def get_session_id (self):
        return self.driver.session_id

    def disable_alert(self):
        self.driver.switch_to.alert.dismiss()

    def element_get_text(self, element, tag):
        if element in self.locator:
            try:
                # Aguardar até que o elemento seja visível e, em seguida, retornar seu texto
                element_text = self.wait.until(EC.visibility_of_element_located((self.locator[element], tag)))
                return element_text
            except TimeoutException:
                print("Elemento não encontrado")   
                  
    def get_elements(self, element, tag):
        if element in self.locator:
            try:
                # Aguardar até que o elemento seja visível e, em seguida, retornar seu texto
                elements = self.wait.until(EC.visibility_of_all_elements_located((self.locator[element], tag)))
                return elements
            except TimeoutException:
                print("Elemento não encontrado")

    def get(self, url):
        # await asyncio.sleep(0)
        self.driver.get(url)
    def close(self):
    #  await asyncio.sleep(0)
        self.driver.quit()   

    def close_session(self, session_id):
        grid_url = "https://grid.consium.com.br/wd/hub"
        session_url = f"{grid_url}/session/{session_id}"
        response = requests.delete(session_url)
        if response.status_code == 200:
            print("Sessão fechada com sucesso!")
        else:
            print("Falha ao fechar a sessão.")

        return response    
    # Funcao para digitar no elemento           
    def sendkeys(self, element, tag, keys):
    #  await asyncio.sleep(0)
        if element in self.locator:
            try:
                self.wait.until(EC.presence_of_element_located((self.locator[element], tag))).send_keys(keys)
            except TimeoutException:
                print("Elemento não encontrado")
                
    # Funcao para clicar no elemento                
    def click(self, element, tag):
    #  await asyncio.sleep(0)
        if element in self.locator:
            try:
                self.wait.until(EC.visibility_of_element_located((self.locator[element], tag))).click()
            except TimeoutException:    
                print("Elemento não encontrado")


    def get_table_element(self, element, tag):
        try:
            # Obter o conteúdo HTML da tag <tbody>
            html_content = self.wait.until(EC.visibility_of_element_located((self.locator[element], tag))).get_attribute('innerHTML')
            # Extrair dados da tabela e transforma em dataframe
            data = self.table_to_dataframe(html_content)
            qtd_linhas = len(data)
            return data, qtd_linhas
        except TimeoutException:
            print("Elemento não encontrado")

    def table_to_dataframe(self, html_content):

        soup = BeautifulSoup(html_content, 'html.parser')

        # Encontra a tabela desejada (selecionando-a pela classe, id ou outras características)
        table = soup.find('table')

        # Verifica se a tabela foi encontrada
        if table:
            # Inicializa uma lista para armazenar os dados da tabela
            table_data = []
            # Itera sobre as linhas da tabela (<tr>)
            for row in table.find_all('tr'):
                # Inicializa uma lista para armazenar os dados de uma linha
                row_data = []
                # Itera sobre as células da linha (<td>)
                for cell in row.find_all(['td']):
                    # Adiciona o texto da célula à lista de dados da linha
                    value = cell.text.strip()
                    # Verifica se o valor não está vazio
                    if value:
                        row_data.append(value)
                    else:
                        row_data.append(None)
                    # Verifica se a célula contém uma tag de âncora (hiperlink)
                    link = cell.find('a')
                    if link:
                        # Se houver uma tag de âncora, adiciona o link (href) à lista de dados da linha
                        row_data.append(link.get('href'))
                    else:
                        row_data.append(None)
                # Adiciona os dados da linha à lista de dados da tabela
                if row_data:
                    table_data.append(row_data)

            # Imprime os dados da tabela
            
            df = pd.DataFrame(table_data)
            df.to_excel('arquivo.xlsx', index=False)

            return df 
        

                   

In [9]:
import sqlite3
import os

class Sqclass:
    def __init__(self):
        db_path = '../database/database.db'
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self.create_table_lists()
        self.create_table_products()
        self.create_table_products_infos()
        
    def close(self):
        self.conn.close()
    
    def create_table_lists(self):
        self.cursor.execute(""" 
                            CREATE TABLE IF NOT EXISTS product_list (
                                id INTEGER PRIMARY KEY AUTOINCREMENT,
                                name TEXT,
                                url TEXT
                            )
                            """)
        self.conn.commit()
    
    def create_table_products(self):
        self.cursor.execute(""" 
                            CREATE TABLE IF NOT EXISTS products (
                                id INTEGER PRIMARY KEY AUTOINCREMENT,
                                title TEXT,
                                photo_src TEXT,
                                product_site_id TEXT,
                                id_list INTEGER,
                                image BLOB,
                                FOREIGN KEY(id_list) REFERENCES product_list(id)
                            )
                            """)
        self.conn.commit()
    
    def create_table_products_infos(self):
        self.cursor.execute(""" 
                            CREATE TABLE IF NOT EXISTS products_infos (
                                id INTEGER PRIMARY KEY AUTOINCREMENT,
                                site_option_id TEXT,
                                size TEXT,
                                price_web REAL,
                                price_b2b REAL,
                                id_product INTEGER,
                                FOREIGN KEY(id_product) REFERENCES products(id)
                            )
                            """)
        self.conn.commit()

    
    def insert_into_product_list(self, name, url):
        self.cursor.execute("""
                            INSERT INTO product_list (name, url)
                            VALUES (?, ?)
                            """, (name, url))
        self.conn.commit()
        return self.cursor.lastrowid  # Retorna o ID do registro inserido
    
    
    def get_product_list(self):
        self.cursor.execute("SELECT * FROM product_list")
        return self.cursor.fetchall()
    
    # Pucha os dados da tabela lists e junta com os dados da tabela 'products' + 'products_infos'
    def get_products(self, id_list):
        self.cursor.execute("""
                            SELECT products.id, products.title, products.photo_src, products.product_site_id, products.image, products_infos.site_option_id, products_infos.size, products_infos.price_web, products_infos.price_b2b
                            FROM products
                            INNER JOIN products_infos
                            ON products.id = products_infos.id_product
                            WHERE products.id_list = ?
                            """, (id_list,))
        return self.cursor.fetchall()

    
    # Método para inserir dados na tabela products verifica se o produto ja existe comparando o product_site_id caso exista faz o update
# Método para inserir ou atualizar dados na tabela 'products'
    def insert_or_update_product(self, title, photo_src, product_site_id, id_list, image_binary):
        # Verificar se o produto já existe
        self.cursor.execute("SELECT id FROM products WHERE product_site_id = ?", (product_site_id,))
        existing_product = self.cursor.fetchone()
        
        if existing_product:
            product_id = existing_product[0]
            # Se o produto existir, faça a atualização
            self.cursor.execute("""
                UPDATE products
                SET title = ?, photo_src = ?, id_list = ?, image = ?
                WHERE product_site_id = ?
            """, (title, photo_src, id_list, image_binary, product_site_id))
        else:
            # Se o produto não existir, faça a inserção
            self.cursor.execute("""
                INSERT INTO products (title, photo_src, product_site_id, id_list, image)
                VALUES (?, ?, ?, ?, ?)
            """, (title, photo_src, product_site_id, id_list, image_binary))
            product_id = self.cursor.lastrowid
        
        # Confirma a transação no banco de dados
        self.conn.commit()
        
        # Retorna o ID do produto existente ou recém-inserido
        return product_id


    # Método para inserir dados na tabela products_infos verifica se as informações ja existe comparando o site_option_id caso exista faz o update
    def insert_or_update_products_infos(self, site_option_id, size, price_web, price_b2b, id_product):
        # Verificar se as informações do produto já existem
        self.cursor.execute("SELECT id FROM products_infos WHERE site_option_id = ?", (site_option_id,))
        existing_info = self.cursor.fetchone()
        
        if existing_info:
            info_id = existing_info[0]
            # Se as informações existirem, faça a atualização
            self.cursor.execute("""
                UPDATE products_infos
                SET size = ?, price_web = ?, price_b2b = ?, id_product = ?
                WHERE site_option_id = ?
            """, (size, price_web, price_b2b, id_product, site_option_id))
        else:
            # Se as informações não existirem, faça a inserção
            self.cursor.execute("""
                INSERT INTO products_infos (site_option_id, size, price_web, price_b2b, id_product)
                VALUES (?, ?, ?, ?, ?)
            """, (site_option_id, size, price_web, price_b2b, id_product))
            info_id = self.cursor.lastrowid
        
        # Confirma a transação no banco de dados
        self.conn.commit()
        
        # Retorna o ID do registro existente ou recém-inserido
        return info_id



In [16]:

import time
import requests
from io import BytesIO
from PIL import Image as PILImage
from selenium.webdriver.support.ui import Select

from tqdm import tqdm  # Importa a biblioteca tqdm

#VARIAVEIS
LOGIN_URL = 'https://b2b.tradeinn.com'
USER_EMAIL = 'vendas@oldfirm.com.br'
USER_PASSWORD = 'Kohlrauschrs18!G'



def login(bot):
        
    bot.get(LOGIN_URL)

    bot.sendkeys('ID', 'email_login', USER_EMAIL)
    bot.sendkeys('ID', 'pass_login', USER_PASSWORD)

    bot.click('XPATH', '/html/body/div[3]/div/div/div[2]/div[1]/form/div[4]/button')

def update_all_products(bot):
    import time

    bd = Sqclass()

    listas = bd.get_product_list()

    for list in listas:
        
        url = list[2]
        LIST_ID = list[0]
        
        get_products_site(LIST_ID, url, bot)
        
    bot.click('XPATH', '/html/body/nav/div/div[4]/div[1]')
    time.sleep(5)
    bot.close()

def download_image(url):
    response = requests.get(url)
    if response.status_code == 200:
        return BytesIO(response.content)
    else:
        raise Exception(f"Failed to download image from {url}")

def image_to_binary(image_stream):
    with PILImage.open(image_stream) as img:
        with BytesIO() as output:
            img.save(output, format='PNG')
            return output.getvalue()

def get_products_site(LIST_ID, url):
    
    bot = Navegador()
    
    login(bot)
    time.sleep(5)
    
    bd = Sqclass()
    
    try:
        bot.get(url)
        
        elements = bot.get_elements('CLASS_NAME', 'product-listing-wrapper')

        total_elements = len(elements)
        
        start_time = time.time()  # Início da medição de tempo

        # Barra de progresso para elementos
        for i, element in enumerate(tqdm(elements, desc="Processando produtos", unit="produto")):
            
            xpath = f"/html/body/div[3]/div[2]/div[1]/div/main/ul/li[{i + 1}]"
            
            bot.click('XPATH', xpath)
            
            title = bot.element_get_text('ID', 'name_product').text
            photo_src = bot.element_get_text('XPATH', '/html/body/div[3]/div[1]/div/div[2]/div[1]/div[2]/div/div[1]/img').get_attribute('src')

            # Baixar e converter a imagem para binário
            try:
                img_stream = download_image(photo_src)
                img_binary = image_to_binary(img_stream)
            except Exception as e:
                print(f"Erro ao baixar ou converter imagem: {e}")
                img_binary = None  # Use None se a imagem não puder ser baixada ou convertida

            product_site_id = bot.element_get_text('XPATH', xpath).get_attribute('data-id_modelo')
            
            # Salvar no BD, incluindo a imagem binária
            new_product = bd.insert_or_update_product(title, photo_src, product_site_id, LIST_ID, img_binary)
            
            # Salvar listas
            drp_element = bot.get_elements('ID', 'tallas_productos')[0]
            select = Select(drp_element)
            num_options = len(select.options)

            # Barra de progresso para tamanhos
            for index in tqdm(range(num_options), desc="Processando tamanhos", unit="tamanho", leave=False):
                
                select.select_by_index(index)
                
                size = select.first_selected_option.text
                option_site_id = select.first_selected_option.get_attribute('value')
                price_web = bot.element_get_text('ID', 'precio_web').text
                price_b2b = bot.element_get_text('ID', 'precio_b2b').text
                
                new_product_info = bd.insert_or_update_products_infos(option_site_id, size, price_web, price_b2b, new_product)

            bot.click('ID', 'js-cerrar-detalle')
            
            # Atualiza a página a cada 8 produtos
            if (i + 1) % 8 == 0:
                bot.get(url)

        end_time = time.time()  # Fim da medição de tempo
        execution_time = end_time - start_time  # Cálculo do tempo de execução 

        print(f"\nTempo de execução: {execution_time:.2f} segundos")
        
    except Exception as e:
        print(e)
        bot.click('XPATH', '/html/body/nav/div/div[4]/div[1]')
        time.sleep(5)
        bot.close()

In [20]:
bot = Navegador()
login(bot)

In [114]:
elements = bot.get_elements('CLASS_NAME', 'ais-InfiniteHits-item')
len(elements)

10

In [117]:
bd = Sqclass()

for i, element in enumerate(tqdm(elements, desc="Processando produtos", unit="produto")):
    
    xpath = f"/html/body/div[3]/div[2]/div[1]/div/main/ol/li[{i + 1}]"
    print(i)
    elements[i].click()
    time.sleep(1)
    bot.click('XPATH', '/html/body/div[3]/div[1]/div/div[1]/div')


Processando produtos:   0%|          | 0/10 [00:00<?, ?produto/s]

0


Processando produtos:  10%|█         | 1/10 [00:01<00:10,  1.17s/produto]

1


Processando produtos:  20%|██        | 2/10 [00:02<00:09,  1.19s/produto]

...

Processando produtos:  80%|████████  | 8/10 [00:09<00:02,  1.24s/produto]

8


Processando produtos:  90%|█████████ | 9/10 [00:11<00:01,  1.27s/produto]

9


Processando produtos: 100%|██████████| 10/10 [00:12<00:00,  1.26s/produto]


In [101]:
product_site_id = bot.element_get_text('ID', 'id_modelo')


Elemento não encontrado


In [104]:
script = """
var element = document.getElementById('id_modelo');
if (element) {
    element.removeAttribute('hidden');  // Remove o atributo 'hidden' se ele existir
    element.setAttribute('type', 'text');  // Altera o tipo para 'text'
}
"""

In [109]:
drp_element = bot.get_elements('ID', 'tallas_productos')
drp_element

[<selenium.webdriver.remote.webelement.WebElement (session="925e5f312c65b0fbf2e8a0242f5bdf88", element="f.72BCDD0318AF1EDB0F8D5A0F62D6F61F.d.5F8956F0F51F7DB7AAD72CC59EC07615.e.20188")>]